In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, cophenet, fcluster, dendrogram
from sklearn.metrics import silhouette_score

In [9]:
# Load CSV files
agg_df = pd.read_csv('pandas/agg_df.csv')
fish_df = pd.read_csv('pandas/fish_df.csv')

In [10]:
#1 Compare linkage+distance via cophenetic correlation
def cophenetic_for(metric, method, X):
    D = pdist(fish_df, metric=metric)
    Z = linkage(D, method=method)
    coph, D_coph = cophenet(Z, D)
    return coph, Z, D

# Try Ward+euclidean(hamming-equivalent for 0/1) and average+jaccard
coph_ward, Z_ward, D_euc = cophenetic_for('euclidean', 'ward', fish_df)
coph_avg,  Z_avg,  D_jac = cophenetic_for('jaccard',   'average', fish_df)
best = ('ward+euclidean', coph_ward, Z_ward, D_euc) if coph_ward >= coph_avg else ('average+jaccard', coph_avg, Z_avg, D_jac)

print(f"Best dendrogram by cophenetic correlation: {best[0]} (coph={best[1]:.3f})")


Best dendrogram by cophenetic correlation: average+jaccard (coph=0.618)


In [14]:

Z = best[2]  # chosen linkage
D = best[3]  # chosen condensed distance

# 3) Mojena stopping rule
# fusion heights are the third column of Z
heights = Z[:,2]
mu, sd = heights.mean(), heights.std()
for c in [1.25, 1.5, 2.0]:
    cut = mu + c*sd
    # Select k by counting merges below cut height
    k = (heights < cut).sum() + 1
    print(f"Mojena c={c}: cut height≈{cut:.3f}, proposed k={k}")
    


Mojena c=1.25: cut height≈0.583, proposed k=2156
Mojena c=1.5: cut height≈0.622, proposed k=2241
Mojena c=2.0: cut height≈0.699, proposed k=2343


In [15]:

# 4) Silhouette across candidate k
for k in range(2, 8):
    labels = fcluster(Z, t=k, criterion='maxclust')
    # Use distance consistent with the chosen method
    metric = 'euclidean' if best[0]=='ward+euclidean' else 'jaccard'
    sil = silhouette_score(squareform(D), labels, metric='precomputed')
    print(f"k={k}: mean silhouette={sil:.3f}")

# 5) (Optional) Bootstrap stability (row bootstraps)
# Repeat: resample rows, recompute labels; build co-association matrix
# ... (omitted here for brevity, but I can run it and summarize stability if you want)

# 6) Re-cluster the “large” cluster with the same rubric:
# Identify the largest cluster at chosen k, subset X, and repeat steps 2–5 on that subset.


k=2: mean silhouette=0.237
k=3: mean silhouette=0.225
k=4: mean silhouette=0.209
k=5: mean silhouette=0.195
k=6: mean silhouette=0.185
k=7: mean silhouette=0.157


In [17]:
import numpy as np

# Patch for dynamicTreeCut compatibility with NumPy 2.0+
if not hasattr(np, 'in1d'):
    np.in1d = np.isin

import pandas as pd
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
from dynamicTreeCut import cutreeHybrid

fish_df_binary = (fish_df > 0).astype(np.int8)

# Jaccard + average linkage dendrogram
D = pdist(fish_df_binary, metric="jaccard")
Z = linkage(D, method="average")

# Hybrid Dynamic Tree Cut
res = cutreeHybrid(Z, D)
labels = res["labels"]

..cutHeight not given, setting it to 0.9844170232033467  ===>  99% of the (truncated) height range in dendro.
..done.


In [20]:
# Display the clustering results from cutreeHybrid
print(f"Number of clusters: {len(np.unique(res['labels']))}")
print(f"Cluster labels (first 10): {res['labels'][:10]}")
print(f"\nCluster distribution:")
unique, counts = np.unique(res['labels'], return_counts=True)
for cluster, count in zip(unique, counts):
	print(f"  Cluster {cluster}: {count} samples")

Number of clusters: 30
Cluster labels (first 10): [ 6  4 26 18 11  2  4  9  4 13]

Cluster distribution:
  Cluster 0: 3 samples
  Cluster 1: 303 samples
  Cluster 2: 238 samples
  Cluster 3: 157 samples
  Cluster 4: 149 samples
  Cluster 5: 146 samples
  Cluster 6: 117 samples
  Cluster 7: 111 samples
  Cluster 8: 111 samples
  Cluster 9: 107 samples
  Cluster 10: 87 samples
  Cluster 11: 86 samples
  Cluster 12: 78 samples
  Cluster 13: 74 samples
  Cluster 14: 71 samples
  Cluster 15: 70 samples
  Cluster 16: 47 samples
  Cluster 17: 45 samples
  Cluster 18: 44 samples
  Cluster 19: 42 samples
  Cluster 20: 40 samples
  Cluster 21: 38 samples
  Cluster 22: 38 samples
  Cluster 23: 37 samples
  Cluster 24: 36 samples
  Cluster 25: 32 samples
  Cluster 26: 29 samples
  Cluster 27: 27 samples
  Cluster 28: 26 samples
  Cluster 29: 23 samples


In [22]:
import numpy as np, pandas as pd
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import silhouette_score, silhouette_samples

# Your labels from DTC (global cut)
labels = res["labels"]            # from cutreeHybrid
D = pdist(fish_df_binary, metric="jaccard")    # same distance used to build Z

# 1) Cluster sizes
sizes = pd.Series(labels).value_counts().sort_values(ascending=False)
print("Total clusters:", sizes.size)
print("Singletons (size=1):", (sizes==1).sum())
print("Clusters ≥10 samples:", (sizes>=10).sum())
display(sizes.head(20))

# 2) Global separation (silhouette with Jaccard)
sil_global = silhouette_score(squareform(D), labels, metric="precomputed")
print(f"Mean silhouette (global): {sil_global:.3f}")

# 3) Per-cluster silhouettes (compute on full distance matrix, then average by cluster)
sil_samples = silhouette_samples(squareform(D), labels, metric="precomputed")
sil_df = pd.DataFrame({"label": labels, "sil": sil_samples})
sil_summary = sil_df.groupby("label")["sil"].mean().sort_values(ascending=False)
print("\nPer-cluster mean silhouette scores:")
display(sil_summary.head(20))


Total clusters: 30
Singletons (size=1): 0
Clusters ≥10 samples: 29


1     303
2     238
3     157
4     149
5     146
6     117
7     111
8     111
9     107
10     87
11     86
12     78
13     74
14     71
15     70
16     47
17     45
18     44
19     42
20     40
Name: count, dtype: int64

Mean silhouette (global): 0.034

Per-cluster mean silhouette scores:


label
0     0.301524
28    0.219241
19    0.181608
16    0.142199
25    0.140994
21    0.111304
15    0.092418
17    0.081688
12    0.073887
11    0.071174
1     0.061726
29    0.056309
18    0.044018
14    0.043245
26    0.041811
3     0.032183
5     0.028776
23    0.026247
22    0.025544
4     0.021996
Name: sil, dtype: float64

In [26]:
import numpy as np, pandas as pd

X = fish_df_binary              # strict binary
labels = res["labels"]                             # from cutreeHybrid

def profile_cluster(k, min_support=0.02, top_n=12):
    idx = np.where(labels==k)[0]
    Xk = X.iloc[idx, :]  # Use .iloc for DataFrame indexing
    species = fish_df_binary.columns

    # 1) Core species by prevalence
    prev = Xk.mean(axis=0)
    core = pd.Series(prev, index=species).sort_values(ascending=False).head(top_n)

    # 2) Pair stats: support, Jaccard, lift (vs global independence)
    Pi = X.mean(axis=0)  # global prevalence baseline for lift
    pairs = []
    m = Xk.shape[1]
    Xk_vals = Xk.values  # Convert to numpy for faster iteration
    for a in range(m):
        xa = Xk_vals[:, a]
        for b in range(a+1, m):
            xb = Xk_vals[:, b]
            both = np.logical_and(xa==1, xb==1).mean()
            either = np.logical_or(xa==1, xb==1).mean()
            jacc = (both / either) if either>0 else 0.0
            exp  = Pi.iloc[a]*Pi.iloc[b]
            lift = (both/exp) if exp>0 else np.nan
            if both >= min_support:
                pairs.append((species[a], species[b], both, jacc, lift))
    pairs_df = pd.DataFrame(pairs, columns=["species_a","species_b","support","jaccard","lift"])\
                 .sort_values(["lift","support"], ascending=[False, False])\
                 .head(top_n)
    return core, pairs_df

# Example: summarize your top clusters by silhouette
for k in [0, 28, 19, 16, 25]:
    core, pairs_top = profile_cluster(k, min_support=0.02)
    print(f"\nCluster {k} — core species (top {len(core)}):\n", core)
    print(f"\nCluster {k} — top pairs by lift (support ≥2%):\n", pairs_top)


Cluster 0 — core species (top 12):
 SMBF    0.666667
BWFN    0.333333
ERSN    0.000000
BLGL    0.000000
GNSF    0.000000
WDSN    0.000000
SVRH    0.000000
LMBS    0.000000
MMSN    0.000000
PGMW    0.000000
SHRH    0.000000
RVSN    0.000000
dtype: float64

Cluster 0 — top pairs by lift (support ≥2%):
 Empty DataFrame
Columns: [species_a, species_b, support, jaccard, lift]
Index: []

Cluster 28 — core species (top 12):
 SHRH    0.884615
RVRH    0.846154
SVRH    0.576923
SMBS    0.576923
FWDM    0.269231
QLBK    0.192308
GDRH    0.076923
WLYE    0.076923
BLGL    0.076923
LMBS    0.076923
BWFN    0.076923
SGER    0.038462
dtype: float64

Cluster 28 — top pairs by lift (support ≥2%):
    species_a species_b   support   jaccard       lift
18      RVRH      QLBK  0.153846  0.173913  31.233862
13      RVRH      SVRH  0.500000  0.541667  28.857857
15      RVRH      SHRH  0.769231  0.800000  24.109924
10      RVRH      SMBS  0.500000  0.541667  18.914326
19      RVRH      FWDM  0.192308  0.2083